In [12]:
from numba import njit, jit, f8
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline


In [13]:
#zb チェック
@jit
def draw_v2d(x, y, z):
    import matplotlib.pyplot as plt
    plt.contourf(x,y,z)
    return 0

In [14]:
# node 
imax = 10000; jmax = 10000

#河床高
zb = np.random.rand(imax, jmax)

#mesh
x, y = np.meshgrid(np.arange(zb.shape[1]), np.arange(zb.shape[0]))

#res = draw_v2d(x, y, zb)

print(zb.shape)
print(zb)

(10000, 10000)
[[0.59181805 0.15817885 0.46750204 ... 0.84934843 0.09564677 0.38158022]
 [0.28551845 0.59302377 0.20209678 ... 0.95344514 0.79910806 0.53640485]
 [0.80612281 0.24392162 0.25294533 ... 0.27927905 0.77208285 0.15278676]
 ...
 [0.73955002 0.59611462 0.51088135 ... 0.49776582 0.36652532 0.87591037]
 [0.61853972 0.41943949 0.50547297 ... 0.36891973 0.85998831 0.96108593]
 [0.05920108 0.56216425 0.19425836 ... 0.82595084 0.50145867 0.35360884]]


In [15]:
#numba jit
@jit("f8[:,:](f8[:,:])", nopython=True)
def func_2(zb):

    dzdx = np.zeros(zb.shape)
    dzdy = np.zeros(zb.shape)
    
    dzdx[1:,0:] = zb[1:, 0:] - zb[0:-1,0:]
  
     
    return dzdx

In [16]:
##diff関数を用いる nは微分の階級（初期値は１）,axis=-1が通常のaxis=1 
##https://deepage.net/features/numpy-diff.html
def func_diff(zb):
    dzdx=np.zeros(zb.shape)
    dzdy=np.zeros(zb.shape)
    dzdx=np.diff(zb,n=1,axis=0)
    

    return dzdx

In [17]:
#numba jit
@jit("f8[:,:](f8[:,:])", nopython=True)
def func_3(zb):
    
    
    dzdx = np.zeros(zb.shape)
    dzdy = np.zeros(zb.shape)
    
    imax = zb.shape[1]
    jmax = zb.shape[0]
    
    for i in range(1,imax):
        for j in range(0,jmax):
            dzdx[i,j] = zb[i,j] - zb[i-1,j]
            
 
    
    return dzdx

In [18]:
print("jit+slice")
t0 = time.time()  
print("start time : ", t0)        
res = func_2(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)
print(res)

jit+slice
start time :  1593595554.8559995
End time :  1593595559.6749399
Elapsed :  4.8189404010772705
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.3062996   0.43484493 -0.26540526 ...  0.1040967   0.70346129
   0.15482462]
 [ 0.52060436 -0.34910215  0.05084855 ... -0.67416609 -0.02702521
  -0.38361809]
 ...
 [ 0.61271864  0.27626688 -0.39391545 ...  0.06938651 -0.57938134
   0.35652487]
 [-0.1210103  -0.17667514 -0.00540838 ... -0.12884609  0.493463
   0.08517556]
 [-0.55933864  0.14272477 -0.31121461 ...  0.45703111 -0.35852965
  -0.60747709]]


In [19]:
print("diff")

t0 = time.time()  
print("start time : ", t0)        
res = func_diff(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)
print(res)

diff
start time :  1593595559.78595
End time :  1593595560.34299
Elapsed :  0.5570399761199951
[[-0.3062996   0.43484493 -0.26540526 ...  0.1040967   0.70346129
   0.15482462]
 [ 0.52060436 -0.34910215  0.05084855 ... -0.67416609 -0.02702521
  -0.38361809]
 [-0.51348827  0.49417983  0.25563596 ...  0.44262845 -0.62278801
   0.32442816]
 ...
 [ 0.61271864  0.27626688 -0.39391545 ...  0.06938651 -0.57938134
   0.35652487]
 [-0.1210103  -0.17667514 -0.00540838 ... -0.12884609  0.493463
   0.08517556]
 [-0.55933864  0.14272477 -0.31121461 ...  0.45703111 -0.35852965
  -0.60747709]]


In [20]:
print("jit+for")
t0 = time.time()  
print("start time : ", t0)        
res = func_3(zb)
t1 = time.time()
print("End time : ", t1)
print("Elapsed : ", t1 - t0)
print(res)

jit+for
start time :  1593595560.366993
End time : 1593595561.481076
Elapsed :  1.1140830516815186
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.3062996   0.43484493 -0.26540526 ...  0.1040967   0.70346129
   0.15482462]
 [ 0.52060436 -0.34910215  0.05084855 ... -0.67416609 -0.02702521
  -0.38361809]
 ...
 [ 0.61271864  0.27626688 -0.39391545 ...  0.06938651 -0.57938134
   0.35652487]
 [-0.1210103  -0.17667514 -0.00540838 ... -0.12884609  0.493463
   0.08517556]
 [-0.55933864  0.14272477 -0.31121461 ...  0.45703111 -0.35852965
  -0.60747709]]


In [21]:
print("jit+slice")
%timeit func_2(zb)
print("diff")
%timeit func_diff(zb)
print("jit+for")
%timeit func_3(zb)

jit+slice
1.65 s ± 138 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
diff
476 ms ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
jit+for
989 ms ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
